In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model_4bit = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-0.5B",
    quantization_config=quantization_config
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
input_text = "What is the capital of France?"
inputs = tokenizer(input_text, return_tensors="pt")
inputs = inputs.to(model_4bit.device)


outputs = model_4bit.generate(**inputs, max_new_tokens=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


What is the capital of France? The capital of France is Paris.


In [ ]:
from peft import prepare_model_for_kbit_training

model_4bit.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model_4bit)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,  # Rank
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4399104 || all params: 319518592 || trainable%: 1.3767912447485997


In [ ]:
from datasets import load_dataset

dataold = load_dataset("Abirate/english_quotes")
dataold = dataold.map(lambda samples: tokenizer(samples["quote"]), batched=True)

README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
import json
from datasets import Dataset

with open("/content/drive/MyDrive/train.json", "r") as f:
    data = json.load(f)["data"]

def generate_prompt(data_point):
  return f"""
<Human>: {data_point["Context"]}
<AI>: {data_point["Response"]}
  """.strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

dataset = Dataset.from_list(data)

dataset = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

In [ ]:
import transformers
OUTPUT_DIR = "/content/drive/MyDrive/Qwen2-0.5B-lora"

trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        learning_rate=2e-4,
        warmup_ratio=0.1,
        fp16=True,
        logging_steps=1,                    # Changed to 1 to see loss at every step
        report_to=["tensorboard"],          # Added tensorboard reporting
        save_strategy="no",
        save_total_limit=1,
        output_dir=OUTPUT_DIR,
        optim="paged_adamw_8bit",
        weight_decay=0.05,
        max_grad_norm=1.0,
        lr_scheduler_type="cosine",
        gradient_checkpointing=True,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False

# Train the model
trainer.train()

# Save only the final checkpoint
trainer.save_model(OUTPUT_DIR)

Step,Training Loss
1,3.893500
2,4.281700
3,3.558700
4,3.586800
5,4.950200
6,3.704700
7,3.818900
8,3.356800
9,2.728900
10,2.575200


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

In [ ]:
def load_model_and_lora(base_model_name="Qwen/Qwen2-0.5B", lora_path="/content/drive/MyDrive/Qwen2-0.5B-lora"):
    # 1. Load base model with quantization
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    # Add padding token if it doesn't exist
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=quantization_config,
        device_map="auto",
        pad_token_id=tokenizer.pad_token_id  # Set pad_token_id in the model config
    )

    # 3. Load LoRA configuration and model
    try:
        config = PeftConfig.from_pretrained(lora_path)
        model = PeftModel.from_pretrained(base_model, lora_path)
    except ValueError as e:
        print(f"Error loading LoRA: {e}")
        print("Falling back to base model only")
        model = base_model

    return model, tokenizer

In [ ]:
def generate_response(model, tokenizer, prompt, max_length=100):
    system_prompt = """You are a Windows CMD command generator. Output only the command, no explanations.

Rules:
1. Use complete paths (C:\\Users\\%USERNAME%\\...)
2. Use quotes for spaces in paths
3. Use proper Windows commands (dir, md, move, copy, del, ren)

Example:
User: "list files in downloads"
Output: dir "C:\\Users\\%USERNAME%\\Downloads"
"""

    # Format prompt
    formatted_prompt = f"{system_prompt}\nUser: {prompt}\nOutput:"

    # Clear cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Tokenize with original max_length
    inputs = tokenizer(formatted_prompt,
                      return_tensors="pt",
                      truncation=True,
                      max_length=512).to(model.device)

    # Generate with strict parameters
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.2,  # Keep temperature low
            top_p=0.9,
            repetition_penalty=1.3,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Clean response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    try:
        # Get only the command
        response = response.split("Output:")[-1].strip()
        response = response.split("\n")[0]  # Take first line only

        # Remove any UI artifacts
        artifacts = ['<', '>', 'COMMAND:', 'Human:', 'AI:', 'User:']
        for artifact in artifacts:
            response = response.replace(artifact, '')

        response = response.strip()

    except:
        return "Error: Could not generate valid command"

    return response

In [ ]:
if __name__ == "__main__":
   # Load model and LoRA
   model, tokenizer = load_model_and_lora()

   # Test generation
   test_questions = [
       "I want to see the contents of my current folder, what to do?",
       "How do I create a new directory?",
       "move the file report.pdf from Downloads to Documents",
       "move all jpg files from Pictures to Backup folder",
      #  "move 'My Summer Vacation.mp4' from Videos to Holiday Memories folder",
      #  "copy config.json from project folder to backup",
      #  "move thesis.docx from Desktop/University/Draft to Documents/Final Thesis",
      #  "create a new folder called Projects in Documents",
      #  "delete temp.txt from Downloads folder",
      #  "rename old_report.pdf to final_report.pdf in Documents folder",
      #  "move presentation.pptx and notes.docx from Desktop to Meeting_Materials folder",
      #  "move all python files from code/src to projects/python/backup"
   ]

   for question in test_questions:
       response = generate_response(model, tokenizer, question)
       print(f"▶️{response}\n")

▶️cd C:\Users\%USERNAME%\downloads

▶️mkdir C:/newdirectory

▶️rename report.pdf C:\Documents and Settings\User\Reports\report.txt

▶️rename *.jpg *Backup/*Pictures/*.jpg

